In [1]:
import cudf
import os

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [3]:
df = cudf.read_parquet('../data/all_states_lat_long_income_education_age_sex.parquet/*')[['GISJOIN', 'p_id', 'sex' ,'age']]

In [4]:
df = df.query('age >= 16')

In [5]:
df['GISJOIN_og'] = df.GISJOIN
df['GISJOIN'] = (df.GISJOIN/1000).astype('int')

In [6]:
df

,GISJOIN,p_id,sex,age,GISJOIN_og
1216,1000100207001,27642,0,23,1000100207001066
1217,1000100207001,27643,0,24,1000100207001066
1218,1000100207001,27644,0,24,1000100207001066
1760,1000100204002,10069,0,40,1000100204002025
1761,1000100204002,10070,0,40,1000100204002025
...,...,...,...,...,...
308745290,56004109752003,308712347,1,63,56004109752003864
308745291,56004109752003,308712348,1,64,56004109752003864
308745292,56004109752003,308712349,1,63,56004109752003864
308745293,56004109752003,308712350,1,64,56004109752003864


In [7]:
df.dtypes

GISJOIN       int64
p_id          int32
sex            int8
age            int8
GISJOIN_og    int64
dtype: object

## Adding cow block wise - Male & Female

<b>JRHE001:</b>    Total</br>
<b>JRHE002:</b>    Male</br>
<b>JRHE003:</b>    Male: Private for-profit wage and salary workers</br>
<b>JRHE004:</b>    Male: Private for-profit wage and salary workers: Employee of private company workers</br>
<b>JRHE005:</b>    Male: Private for-profit wage and salary workers: Self-employed in own incorporated business workers</br>
<b>JRHE006:</b>    Male: Private not-for-profit wage and salary workers</br>
<b>JRHE007:</b>    Male: Local government workers</br>
<b>JRHE008:</b>    Male: State government workers</br>
<b>JRHE009:</b>    Male: Federal government workers</br>
<b>JRHE010:</b>    Male: Self-employed in own not incorporated business workers</br>
<b>JRHE011:</b>    Male: Unpaid family workers</br>
<b>JRHE012:</b>    Female</br>
<b>JRHE013:</b>    Female: Private for-profit wage and salary workers</br>
<b>JRHE014:</b>    Female: Private for-profit wage and salary workers: Employee of private company workers</br>
<b>JRHE015:</b>    Female: Private for-profit wage and salary workers: Self-employed in own incorporated business workers</br>
<b>JRHE016:</b>    Female: Private not-for-profit wage and salary workers</br>
<b>JRHE017:</b>    Female: Local government workers</br>
<b>JRHE018:</b>    Female: State government workers</br>
<b>JRHE019:</b>    Female: Federal government workers</br>
<b>JRHE020:</b>    Female: Self-employed in own not incorporated business workers</br>
<b>JRHE021:</b>    Female: Unpaid family workers</br>

In [8]:
df_cat = cudf.read_csv('/home/ajay/data/census/all_states_block_group/nhgis0016_csv/nhgis0016_ds176_20105_2010_blck_grp.csv')

df_cat.STATEA = df_cat.STATEA.astype('int')
df_cat = df_cat.query('STATEA != 72')

df_cat.GISJOIN = df_cat.GISJOIN.str.replace('G', '').astype('int')

sex_by_cow_cols = ['JRHE002','JRHE004','JRHE005','JRHE006','JRHE007','JRHE008','JRHE009','JRHE010','JRHE011','JRHE012','JRHE014','JRHE015','JRHE016','JRHE017','JRHE018','JRHE019','JRHE020','JRHE021']

sex_by_cow_cols_male = ['JRHE002','JRHE004','JRHE005','JRHE006','JRHE007','JRHE008','JRHE009','JRHE010','JRHE011']
sex_by_cow_cols_female = ['JRHE012','JRHE014','JRHE015','JRHE016','JRHE017','JRHE018','JRHE019','JRHE020','JRHE021']

df_cat = df_cat[['GISJOIN']+sex_by_cow_cols]

for i in df_cat.columns:
    if i != 'GISJOIN':
        df_cat[i] = df_cat[i].astype('int16')
        
df_cat = df_cat.fillna(0)

In [9]:
df_cat

,GISJOIN,JRHE002,JRHE004,JRHE005,JRHE006,JRHE007,JRHE008,JRHE009,JRHE010,JRHE011,JRHE012,JRHE014,JRHE015,JRHE016,JRHE017,JRHE018,JRHE019,JRHE020,JRHE021
0,1000100201001,85,45,0,0,13,17,10,0,0,162,89,16,13,13,21,0,10,0
1,1000100201002,342,203,0,0,51,39,38,11,0,326,205,10,9,0,36,46,20,0
2,1000100202001,154,129,0,0,0,13,0,12,0,238,191,0,0,28,0,19,0,0
3,1000100202002,200,77,0,12,27,25,49,10,0,202,162,0,0,29,11,0,0,0
4,1000100203001,701,497,4,25,13,107,42,13,0,528,392,0,28,41,44,19,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217735,56004509511001,435,328,4,26,18,3,7,40,9,301,144,4,40,25,26,11,33,18
217736,56004509511002,475,294,6,0,56,19,21,79,0,282,196,0,37,42,0,0,7,0
217737,56004509513001,445,339,59,0,14,0,33,0,0,341,142,76,0,39,58,26,0,0
217738,56004509513002,283,128,14,4,34,39,13,51,0,216,151,0,9,26,17,10,3,0


In [10]:
all(df_cat[sex_by_cow_cols_male[1:]].to_pandas().sum(axis=1).values == df_cat.JRHE002.to_pandas().values)

True

In [11]:
all(df_cat[sex_by_cow_cols_female[1:]].to_pandas().sum(axis=1).values == df_cat.JRHE012.to_array())

True

In [12]:
for i in sex_by_cow_cols_male:
    if i != 'JRHE002':
        df_cat[i] = df_cat[i]/df_cat['JRHE002'].astype('float32')#*100).floor().astype('int16')

for i in sex_by_cow_cols_female:
    if i != 'JRHE012':
        df_cat[i] = df_cat[i]/df_cat['JRHE012'].astype('float32')#*100).floor().astype('int16')
        
df_cat.drop_column('JRHE002')
df_cat.drop_column('JRHE012')

In [13]:
df_cat

,GISJOIN,JRHE004,JRHE005,JRHE006,JRHE007,JRHE008,JRHE009,JRHE010,JRHE011,JRHE014,JRHE015,JRHE016,JRHE017,JRHE018,JRHE019,JRHE020,JRHE021
0,1000100201001,0.529412,0.000000,0.000000,0.152941,0.200000,0.117647,0.000000,0.00000,0.549383,0.098765,0.080247,0.080247,0.129630,0.000000,0.061728,0.000000
1,1000100201002,0.593567,0.000000,0.000000,0.149123,0.114035,0.111111,0.032164,0.00000,0.628834,0.030675,0.027607,0.000000,0.110429,0.141104,0.061350,0.000000
2,1000100202001,0.837662,0.000000,0.000000,0.000000,0.084416,0.000000,0.077922,0.00000,0.802521,0.000000,0.000000,0.117647,0.000000,0.079832,0.000000,0.000000
3,1000100202002,0.385000,0.000000,0.060000,0.135000,0.125000,0.245000,0.050000,0.00000,0.801980,0.000000,0.000000,0.143564,0.054455,0.000000,0.000000,0.000000
4,1000100203001,0.708987,0.005706,0.035663,0.018545,0.152639,0.059914,0.018545,0.00000,0.742424,0.000000,0.053030,0.077652,0.083333,0.035985,0.007576,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217735,56004509511001,0.754023,0.009195,0.059770,0.041379,0.006897,0.016092,0.091954,0.02069,0.478405,0.013289,0.132890,0.083056,0.086379,0.036545,0.109635,0.059801
217736,56004509511002,0.618947,0.012632,0.000000,0.117895,0.040000,0.044211,0.166316,0.00000,0.695035,0.000000,0.131206,0.148936,0.000000,0.000000,0.024823,0.000000
217737,56004509513001,0.761798,0.132584,0.000000,0.031461,0.000000,0.074157,0.000000,0.00000,0.416422,0.222874,0.000000,0.114370,0.170088,0.076246,0.000000,0.000000
217738,56004509513002,0.452297,0.049470,0.014134,0.120141,0.137809,0.045936,0.180212,0.00000,0.699074,0.000000,0.041667,0.120370,0.078704,0.046296,0.013889,0.000000


### merge df_cat's index with df

In [14]:
df = df.merge(df_cat[['GISJOIN']], on='GISJOIN')

In [15]:
df

,GISJOIN,p_id,sex,age,GISJOIN_og
0,1000100209001,43848,0,32,1000100209001066
1,1000100209001,43849,0,34,1000100209001066
2,1000100209001,43850,0,30,1000100209001066
3,1000100209001,43851,0,33,1000100209001066
4,1000100208022,36252,0,18,1000100208022024
...,...,...,...,...,...
243227903,56004509513002,308742844,1,72,56004509513002024
243227904,56004509513002,308742845,1,72,56004509513002024
243227905,56004509513002,308742846,1,71,56004509513002024
243227906,56004509513002,308742847,1,74,56004509513002024


#### Make sure total ratio is a complete 1 (or 0 incase there are no values)

In [16]:
df1 = df.groupby(by=['GISJOIN', 'sex'], as_index=False).count()

In [17]:
df1 = df1.one_hot_encoding('sex', 'gender', [0,1])

In [18]:
df1.drop_column('sex')

In [19]:
df1.gender_0 = (df1.GISJOIN_og*df1.gender_0).astype('int')
df1.gender_1 = (df1.GISJOIN_og*df1.gender_1).astype('int')

In [20]:
df1.drop_column('GISJOIN_og')

In [21]:
df1 = df1.groupby('GISJOIN', as_index=False).sum()
df1

,GISJOIN,p_id,age,gender_0,gender_1
0,1000100201001,575,575,281,294
1,1000100201002,929,929,423,506
2,1000100202001,734,734,292,442
3,1000100202002,921,921,483,438
4,1000100203001,1840,1840,739,1101
...,...,...,...,...,...
216859,56004509511001,1279,1279,698,581
216860,56004509511002,1548,1548,906,642
216861,56004509513001,943,943,402,541
216862,56004509513002,1118,1118,518,600


#### Merge population in df with ratios in df_cat

In [22]:
df1 = df1.merge(df_cat, on='GISJOIN')

In [23]:
df1

,GISJOIN,p_id,age,gender_0,gender_1,JRHE004,JRHE005,JRHE006,JRHE007,JRHE008,...,JRHE010,JRHE011,JRHE014,JRHE015,JRHE016,JRHE017,JRHE018,JRHE019,JRHE020,JRHE021
0,6003701190023,988,988,502,486,0.742308,0.046154,0.000000,0.000000,0.000000,...,0.211538,0.000000,0.517483,0.000000,0.069930,0.083916,0.000000,0.000000,0.328671,0.000000
1,6003701190024,1414,1414,827,587,0.785463,0.000000,0.053927,0.018757,0.000000,...,0.141852,0.000000,0.680387,0.000000,0.000000,0.186441,0.000000,0.000000,0.133172,0.000000
2,6003701190025,1060,1060,531,529,0.631714,0.094629,0.000000,0.132992,0.000000,...,0.140665,0.000000,0.625000,0.000000,0.075000,0.100000,0.056250,0.093750,0.050000,0.000000
3,6003701192011,926,926,493,433,0.730198,0.000000,0.099010,0.000000,0.032178,...,0.138614,0.000000,0.527473,0.113553,0.146520,0.000000,0.000000,0.080586,0.131868,0.000000
4,6003701192012,850,850,514,336,0.816901,0.000000,0.014085,0.138028,0.000000,...,0.030986,0.000000,0.688995,0.000000,0.023923,0.229665,0.000000,0.000000,0.057416,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216859,55000709603003,1111,1111,558,553,0.453731,0.119403,0.098507,0.041791,0.077612,...,0.140299,0.000000,0.382979,0.077128,0.071809,0.257979,0.005319,0.117021,0.087766,0.000000
216860,55000709604001,1228,1228,612,616,0.604072,0.072398,0.054299,0.070136,0.013575,...,0.158371,0.013575,0.585427,0.045226,0.095477,0.153266,0.017588,0.012563,0.087940,0.002513
216861,55000709604002,582,582,331,251,0.573427,0.041958,0.000000,0.195804,0.027972,...,0.160839,0.000000,0.617450,0.033557,0.120805,0.053691,0.020134,0.000000,0.154362,0.000000
216862,55000709604003,722,722,363,359,0.483333,0.079167,0.054167,0.058333,0.066667,...,0.241667,0.000000,0.704225,0.000000,0.098592,0.070423,0.037559,0.046948,0.023474,0.018779


In [24]:
#males
df1.to_pandas().query('JRHE004.isnull()')

,GISJOIN,p_id,age,gender_0,gender_1,JRHE004,JRHE005,JRHE006,JRHE007,JRHE008,...,JRHE010,JRHE011,JRHE014,JRHE015,JRHE016,JRHE017,JRHE018,JRHE019,JRHE020,JRHE021
1222,4002100014053,298,298,132,166,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.000000,0.0,0.0000,0.000000,0.0000,0.000,0.000000,0.0
1414,4000909616003,1320,1320,1320,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1432,4001200202013,222,222,92,130,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.358974,0.0,0.0000,0.051282,0.0000,0.000,0.589744,0.0
1574,6003705041021,22,22,0,22,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1685,2006000001002,16,16,0,16,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214055,54006100101012,176,176,0,176,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000,0.0,0.3125,0.000000,0.3125,0.375,0.000000,0.0
214910,51005909801001,10,10,0,10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214911,51005909803001,4,4,0,4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215007,48011300027011,518,518,140,378,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000,0.0,0.0000,1.000000,0.0000,0.000,0.000000,0.0


In [25]:
#females
df1.to_pandas().query('JRHE014.isnull()')

,GISJOIN,p_id,age,gender_0,gender_1,JRHE004,JRHE005,JRHE006,JRHE007,JRHE008,...,JRHE010,JRHE011,JRHE014,JRHE015,JRHE016,JRHE017,JRHE018,JRHE019,JRHE020,JRHE021
56,4001509515014,982,982,493,489,0.638554,0.0,0.156627,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1414,4000909616003,1320,1320,1320,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1574,6003705041021,22,22,0,22,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1685,2006000001002,16,16,0,16,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1873,6003705409024,1,1,0,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214910,51005909801001,10,10,0,10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214911,51005909803001,4,4,0,4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215144,51013509801001,34,34,0,34,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216424,51081000450001,1100,1100,998,102,0.406250,0.0,0.000000,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df2 = df1.copy()

In [27]:
null_replace_equal_distribute_factor = (100/len(sex_by_cow_cols_male[1:]))/100
null_replace_equal_distribute_factor

0.125

In [28]:
#males
for i in sex_by_cow_cols_male[1:]:
    df1[i] = ((df1[i] * df1.gender_0)).floor().astype('int16')
    df2[i] = ((df2[i] * df2.gender_0) - df1[i]).astype('float32')
    
t = df2.to_pandas().query('JRHE004.isnull()').index #everything is null for a row in this case

for i in sex_by_cow_cols_male[1:]:
    df1[i].loc[[t]] = 0
    df2[i].loc[[t]] = null_replace_equal_distribute_factor

df1['diff_male'] = df1.gender_0.to_pandas() - df1[sex_by_cow_cols_male[1:]].to_pandas().sum(axis=1)

In [29]:
#females
# df2 = df1.copy()
for i in sex_by_cow_cols_female[1:]:
    df1[i] = ((df1[i] * df1.gender_1)).floor().astype('int16')
    df2[i] = ((df2[i] * df2.gender_1) - df1[i]).astype('float32')
    
t = df2.to_pandas().query('JRHE014.isnull()').index #everything is null for a row in this case

for i in sex_by_cow_cols_female[1:]:
    df1[i].loc[[t]] = 0
    df2[i].loc[[t]] = null_replace_equal_distribute_factor

df1['diff_female'] = df1.gender_1.to_pandas() - df1[sex_by_cow_cols_female[1:]].to_pandas().sum(axis=1)

In [30]:
df1

,GISJOIN,p_id,age,gender_0,gender_1,JRHE004,JRHE005,JRHE006,JRHE007,JRHE008,...,JRHE014,JRHE015,JRHE016,JRHE017,JRHE018,JRHE019,JRHE020,JRHE021,diff_male,diff_female
0,6003701190023,988,988,502,486,372,23,0,0,0,...,251,0,33,40,0,0,159,0,1,3
1,6003701190024,1414,1414,827,587,649,0,44,15,0,...,399,0,0,109,0,0,78,0,2,1
2,6003701190025,1060,1060,531,529,335,50,0,70,0,...,330,0,39,52,29,49,26,0,2,4
3,6003701192011,926,926,493,433,359,0,48,0,15,...,228,49,63,0,0,34,57,0,3,2
4,6003701192012,850,850,514,336,419,0,7,70,0,...,231,0,8,77,0,0,19,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216859,55000709603003,1111,1111,558,553,253,66,54,23,43,...,211,42,39,142,2,64,48,0,3,5
216860,55000709604001,1228,1228,612,616,369,44,33,42,8,...,360,27,58,94,10,7,54,1,4,5
216861,55000709604002,582,582,331,251,189,13,0,64,9,...,154,8,30,13,5,0,38,0,3,3
216862,55000709604003,722,722,363,359,175,28,19,21,24,...,252,0,35,25,13,16,8,6,3,4


In [31]:
df1.diff_male.max()

12108

In [32]:
df2[sex_by_cow_cols_male[1:]].to_pandas().sum(axis=1).unique()

array([0.99998692, 2.00002   , 2.00000794, ..., 2.00000088, 4.99999443,
       3.99999476])

In [33]:
print(df1.diff_male.sum() == (df1.gender_0.to_array().sum() - df1[sex_by_cow_cols_male[1:]].to_pandas().sum(axis=1).values.sum()))
print(df1.diff_female.sum() == (df1.gender_1.to_array().sum() - df1[sex_by_cow_cols_female[1:]].to_pandas().sum(axis=1).values.sum()))

True
True


In [34]:
print(df1.gender_0.to_array().sum() == df1[sex_by_cow_cols_male[1:]].to_pandas().sum(axis=1).values.sum())
print(df1.gender_1.to_array().sum() == df1[sex_by_cow_cols_female[1:]].to_pandas().sum(axis=1).values.sum())

False
False


#### filling the difference

Since they don't add up to the total population, lets apply the largest remainder method, mentioned here -> https://revs.runtime-revolution.com/getting-100-with-rounded-percentages-273ffa70252b

In [35]:
import cupy

cp_array_male = cupy.array(df2[sex_by_cow_cols_male[1:]].to_gpu_matrix())
cp_array_male = cupy.nan_to_num(cp_array_male/ cp_array_male.sum(axis=1, keepdims=True))
cp_array_male_sorted = cupy.asnumpy(cp_array_male.argsort())
cp_array_male = cupy.asnumpy(cp_array_male)

cp_array_female = cupy.array(df2[sex_by_cow_cols_female[1:]].to_gpu_matrix())
cp_array_female = cupy.nan_to_num(cp_array_female/ cp_array_female.sum(axis=1, keepdims=True))
cp_array_female_sorted = cupy.asnumpy(cp_array_female.argsort())
cp_array_female = cupy.asnumpy(cp_array_female)

Handle remaining remainder values

In [36]:
from multiprocessing import  Pool
from functools import partial
import numpy as np
import pandas as pd

def parallelize(data, func, num_of_processes=8):
    data_split = np.array_split(data, num_of_processes)
    pool = Pool(num_of_processes)
    data = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return data

def run_on_subset(func, data_subset):
    return data_subset.apply(func, axis=1)

def parallelize_on_rows(data, func, num_of_processes=8):
    return parallelize(data, partial(run_on_subset, func), num_of_processes)

In [37]:
df1_pandas = df1.to_pandas()

count = 0

def applyrows(row):
    global count, cp_array_male_sorted, cp_array_male, cp_array_female_sorted, cp_array_female
    #males
    sorted_series_male = cp_array_male_sorted[row.name][::-1]
    i = row['diff_male']
    while i > 0:
        for sorted_index in sorted_series_male:
            sorted_val = cp_array_male[row.name][sorted_index]
            if sorted_val == 0:
                break
            if int(sorted_val * i) <= 1:
                t_value = i
            else:
                t_value = int(sorted_val * i)
            t_index = 'JRHE0'+str(sorted_index+4).zfill(2)
            row[t_index] += t_value
            i -= t_value
    #females
    sorted_series_female = cp_array_female_sorted[row.name][::-1]
    i = row['diff_female']
    while i > 0:
        for sorted_index in sorted_series_female:
            sorted_val = cp_array_female[row.name][sorted_index]
            if sorted_val == 0:
                break
            if int(sorted_val * i) <= 1:
                t_value = i
            else:
                t_value = int(sorted_val * i)
            t_index = 'JRHE0'+str(sorted_index+14).zfill(2)
            row[t_index] += t_value
            i -= t_value
    
    count += 1
    if count%10000 == 0: 
        print('                                  ', end='\r')
        print('count', count,row.name, end='\r')
    return row

# df2_pandas = df1_pandas.apply(applyrows, axis=1)
df2_pandas = parallelize_on_rows(df1_pandas, applyrows) 

In [38]:
df2_pandas

,GISJOIN,p_id,age,gender_0,gender_1,JRHE004,JRHE005,JRHE006,JRHE007,JRHE008,...,JRHE014,JRHE015,JRHE016,JRHE017,JRHE018,JRHE019,JRHE020,JRHE021,diff_male,diff_female
0,6003701190023,988,988,502,486,373,23,0,0,0,...,251,0,36,40,0,0,159,0,1,3
1,6003701190024,1414,1414,827,587,649,0,46,15,0,...,399,0,0,110,0,0,78,0,2,1
2,6003701190025,1060,1060,531,529,335,50,0,70,0,...,330,0,39,56,29,49,26,0,2,4
3,6003701192011,926,926,493,433,362,0,48,0,15,...,228,49,63,0,0,36,57,0,3,2
4,6003701192012,850,850,514,336,419,0,7,73,0,...,232,0,8,77,0,0,19,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216859,55000709603003,1111,1111,558,553,253,66,57,23,43,...,211,42,39,142,7,64,48,0,3,5
216860,55000709604001,1228,1228,612,616,369,44,33,42,8,...,360,32,58,94,10,7,54,1,4,5
216861,55000709604002,582,582,331,251,189,16,0,64,9,...,157,8,30,13,5,0,38,0,3,3
216862,55000709604003,722,722,363,359,175,31,19,21,24,...,252,0,35,25,13,20,8,6,3,4


In [39]:
print(df2_pandas.gender_0.values.sum() == df2_pandas[sex_by_cow_cols_male[1:]].sum(axis=1).values.sum())
print(df2_pandas.gender_1.values.sum() == df2_pandas[sex_by_cow_cols_female[1:]].sum(axis=1).values.sum())

True
True


In [40]:
df_final = cudf.from_pandas(df2_pandas)

In [41]:
df_final

,GISJOIN,p_id,age,gender_0,gender_1,JRHE004,JRHE005,JRHE006,JRHE007,JRHE008,...,JRHE014,JRHE015,JRHE016,JRHE017,JRHE018,JRHE019,JRHE020,JRHE021,diff_male,diff_female
0,6003701190023,988,988,502,486,373,23,0,0,0,...,251,0,36,40,0,0,159,0,1,3
1,6003701190024,1414,1414,827,587,649,0,46,15,0,...,399,0,0,110,0,0,78,0,2,1
2,6003701190025,1060,1060,531,529,335,50,0,70,0,...,330,0,39,56,29,49,26,0,2,4
3,6003701192011,926,926,493,433,362,0,48,0,15,...,228,49,63,0,0,36,57,0,3,2
4,6003701192012,850,850,514,336,419,0,7,73,0,...,232,0,8,77,0,0,19,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216859,55000709603003,1111,1111,558,553,253,66,57,23,43,...,211,42,39,142,7,64,48,0,3,5
216860,55000709604001,1228,1228,612,616,369,44,33,42,8,...,360,32,58,94,10,7,54,1,4,5
216861,55000709604002,582,582,331,251,189,16,0,64,9,...,157,8,30,13,5,0,38,0,3,3
216862,55000709604003,722,722,363,359,175,31,19,21,24,...,252,0,35,25,13,20,8,6,3,4


In [42]:
print(df_final.gender_0.to_array().sum() == df_final[sex_by_cow_cols_male[1:]].to_pandas().sum(axis=1).values.sum())
print(df_final.gender_1.to_array().sum() == df_final[sex_by_cow_cols_female[1:]].to_pandas().sum(axis=1).values.sum())

True
True


In [43]:
df_final.drop_column('diff_male')
df_final.drop_column('diff_female')
# df_final.drop_column('GISJOIN')
# df_final.rename({'GISJOIN_og':'GISJOIN'}, inplace=True)

In [44]:
df_final.to_parquet('cow_step_1_final.parquet')

/home/ajay/anaconda3/envs/rapids_dev_0.12/lib/python3.6/site-packages/cudf/io/parquet.py:70: UserWarning: Using CPU via PyArrow to write Parquet dataset, this will be GPU accelerated in the future
  "Using CPU via PyArrow to write Parquet dataset, this will "


In [45]:
df.to_parquet('GISJOIN_to_GISJOIN_og_mapping.parquet')